# Data Science Homework 4
Try different imbalanced classification datasets using $k$-fold cross validation and various classification methods.
TODO List:
- Make sure we can open all the data as either DataFrame or nparray
- Handle categorical data (tokenize, one-hot encoding, ....)
- Split each dataset into training and testing dataset.
- Perform any necessary sampling, imputaiton, encoding techniques depending on dataset
- Perform 5-fold cross-validation to select datasets.

In [1]:
import os
import numpy as np
import pandas as pd
import sklearn.metrics as metrics
import category_encoders as ce # sklearn library
import xgboost

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, KFold
from sklearn import preprocessing

DATA_DIR = './data'
files = os.listdir(DATA_DIR)
if '.ipynb_checkpoints' in files:
    files.remove('.ipynb_checkpoints')

special_delims = { 'arcene_train.data': ' '}
no_headers = ['covtype.data', 'arcene_train.data']

print(files)

['abalone.data', 'arcene_train.data', 'biodeg.csv', 'covtype.data', 'dataset', 'eeg_eye_state.arff', 'heart_2020_cleaned.csv', 'HTRU_2.csv', 'income_evaluation.csv', 'phpAmSP4g.arff', 'Raisin_Dataset.arff', 'spambase.data', 'UCI_Credit_Card.csv', 'WA_Fn-UseC_-Telco-Customer-Churn.csv', 'WineQT.csv']


## Load datasets
Some datasets are in csv format, others have just the data. First convert to `DataFrame`s to allow for numeric, categorical values.

In [2]:

def load(name, header=True):
    sep = special_delims[name] if name in special_delims else ','
    name = os.path.join(DATA_DIR, name)
    if header:
        df = pd.read_csv(name, sep=sep)
    else:
        df = pd.read_csv(name, header=None, sep=sep)
    return df

## Encode Categorical Values

In [3]:
def encode_categorical(data:pd.DataFrame, method='ordinal'):
    if method == 'ordinal':
        encoder = ce.OrdinalEncoder()
    elif method == 'onehot':
        encoder = ce.OneHotEncoder()
    elif method == 'label':
        encoder = preprocessing.LabelEncoder()
    return encoder.fit_transform(data)

def scale_features(data:pd.DataFrame, method='standard') -> pd.DataFrame:
    if method == 'standard':
        scaler = preprocessing.StandardScaler()
    elif method == 'minmax':
        scaler = preprocessing.MinMaxScaler()
    return scaler.fit_transform(data)

## Train model:
The evalutation should be fixed on 5-fold cross validation, choose from `RandomForest`, `GBDT`, `XGBoost`,`LightBGM`, `CatBoost`, `KNN`, `Logistic Regression`,`MLP`, `SVM`.

Train a new model every iteration of the cross-validation.

In [4]:

def get_model(method):
    if method == 'xgboost':
        model = xgboost.XGBClassifier(5)
    elif method == 'knn':
        model = KNeighborsClassifier()
    elif method == 'forest':
        model = RandomForestClassifier(10)
    else:
        print(f'{method} not supported.')
        model = None
    return model

def get_metric(method):
    if method == 'acc':
        met = metrics.accuracy_score
    elif method == 'auc':
        met = metrics.auc
    elif method == 'roc_auc':
        met = metrics.roc_auc_score
    elif method == 'f1':
        met = metrics.f1_score
    return met

def train(x, y, method='xgboost', metric='acc'):
    # Perform K-Fold cross validation
    metric = get_metric(metric)
    n_items = x.shape[0]
    kf = KFold( n_splits=5, shuffle=True)
    
    mean_score = 0 
    for i, (train_index, test_index) in enumerate(kf.split(x)):
        model = get_model(method)
        x_train, x_test = x.iloc[train_index,: ], x.iloc[test_index, :]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        if method == 'xgboost':
            y_train -= 1 # Class labels should be zero-based
        
        model.fit(x_train, y_train)
        
        # Evaluate
        preds = model.predict(x_test)
        score = metric(y_test, preds)
        mean_score += score
        #print(f'\t{mean_score}')
        
    mean_score /= (i+1)
    #print(f'Mean score over validation: {mean_score} using {method}')

    return mean_score
        

## P1: How does feature scaling (e.g. performing normalization) affect performance?
With standardization we apply the formula
$$x' = \frac{x - \mu}{\sigma}$$
so that we have 0 mean in the training data.

For One-hot Encoding we should not use this with tree-based models. For these models we can use label encoding, feature-hashing.
We should first encode the categorical values, then train the models on all our datasets both with feature scaling and without.

In [ ]:
for file in files:
    
    df = load(file, False if file in no_headers else True)
    # Encode categorical columns
    cats = df.select_dtypes(include=['object'])
    df[cats.columns] = encode_categorical(cats)
    
    control = df.copy(deep=True) # Compare with standardized dataset

    df, labels = df.iloc[:, :-1], df.iloc[:, -1]
    control, c_labels = control.iloc[:, :-1], control.iloc[:, -1]
    
    # Feature scaling
    df = pd.DataFrame(scale_features(df))
    
    # Train model
    print(f'Training on {file}')
    score = train(df, labels, method='knn', metric='acc')
    print(f'train: {file}: avg: {score}')
   
    score = train(control, c_labels, method='knn', metric='acc')
    print(f'test: {file}: avg: {score}')
    print('\n')
    # The performance is mostly the same across models

## P2 When using tree-based algorithms, will usng one-hot encoding for categorical features generate worse performance than using label encoding? Why?
One-hot encoding transforms inputs with a range of $(1,\dots,n)$. This method generates extra columns and usually results in a sparse matrix.

Label Encoding adds replaces each unique class name with an integer.

In [8]:
for file in files:
    df = load(file, False if file in no_headers else True)
    target_column = df.columns[-1]
    label = df.copy(deep=True) # Use label encoding
    
    cats = df.select_dtypes(include=['object'])

    # Ignore files w/ no categorical data, otherw encoder error
    if cats.empty:
        print(f'No categorical in {file}\n')
        continue
        
    n_unique = df[target_column].unique().shape[0] # Num of unique items in target column

    one_hot = pd.get_dummies(df)
    cats = label.select_dtypes(include=['object'])
    #print(f'{file}: {target_column}')
    # Apply Label Encoder to each column
    for column in cats.columns:
        label[column] = encode_categorical(label[column], method='label')
        
    # If target column is not categorical, labels will only be one column
    if df.dtypes[target_column] == object:
        n_unique = df[target_column].unique().shape[0]
    else:
        n_unique = 1
    
    x_ohe, y_ohe = one_hot.iloc[:,:-n_unique], one_hot.iloc[:,-n_unique:]
    x_label, y_label = label.iloc[:,:-1], label.iloc[:,-1]
    
    #print(f'{file}: xcols: {len(x_label)} ycols: {len(y_label)}')
    #print(one_hot)
    print(f'Training on {file}')
    score = train(x_ohe, y_ohe, method='forest', metric='acc')
    print(f'train: {file}: avg: {score}')
    
    score = train(x_label, y_label, method='forest', metric='acc')
    print(f'test: {file}: avg: {score}')
    print('\n')
    

Training on abalone.data
train: abalone.data: avg: 1.0


C:\Users\Andres Ponce\AppData\Local\Temp\ipykernel_12160\1500941184.py:38: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x_train, y_train)
C:\Users\Andres Ponce\AppData\Local\Temp\ipykernel_12160\1500941184.py:38: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x_train, y_train)
C:\Users\Andres Ponce\AppData\Local\Temp\ipykernel_12160\1500941184.py:38: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x_train, y_train)
C:\Users\Andres Ponce\AppData\Local\Temp\ipykernel_12160\1500941184.py:38: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using

test: abalone.data: avg: 0.23558146864166404


No categorical in arcene_train.data

Training on biodeg.csv
train: biodeg.csv: avg: 0.8360189573459715
test: biodeg.csv: avg: 0.8511848341232227


No categorical in covtype.data

Training on dataset
train: dataset: avg: 0.9883889450623524


C:\Users\Andres Ponce\AppData\Local\Temp\ipykernel_12160\1500941184.py:38: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x_train, y_train)
C:\Users\Andres Ponce\AppData\Local\Temp\ipykernel_12160\1500941184.py:38: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x_train, y_train)
C:\Users\Andres Ponce\AppData\Local\Temp\ipykernel_12160\1500941184.py:38: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x_train, y_train)
C:\Users\Andres Ponce\AppData\Local\Temp\ipykernel_12160\1500941184.py:38: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using

test: dataset: avg: 0.8763835524098417


No categorical in eeg_eye_state.arff

Index(['BMI', 'PhysicalHealth', 'MentalHealth', 'SleepTime', 'HeartDisease_No',
       'HeartDisease_Yes', 'Smoking_No', 'Smoking_Yes', 'AlcoholDrinking_No',
       'AlcoholDrinking_Yes', 'Stroke_No', 'Stroke_Yes', 'DiffWalking_No',
       'DiffWalking_Yes', 'Sex_Female', 'Sex_Male', 'AgeCategory_18-24',
       'AgeCategory_25-29', 'AgeCategory_30-34', 'AgeCategory_35-39',
       'AgeCategory_40-44', 'AgeCategory_45-49', 'AgeCategory_50-54',
       'AgeCategory_55-59', 'AgeCategory_60-64', 'AgeCategory_65-69',
       'AgeCategory_70-74', 'AgeCategory_75-79', 'AgeCategory_80 or older',
       'Race_American Indian/Alaskan Native', 'Race_Asian', 'Race_Black',
       'Race_Hispanic', 'Race_Other', 'Race_White', 'Diabetic_No',
       'Diabetic_No, borderline diabetes', 'Diabetic_Yes',
       'Diabetic_Yes (during pregnancy)', 'PhysicalActivity_No',
       'PhysicalActivity_Yes', 'GenHealth_Excellent', 'GenHealth